In [1]:
import os
import sys
sys.path.append(os.pardir)
import numpy as np
from numpy.random import *
from scipy import ceil, complex64, float64, hamming, zeros
from matplotlib import pylab as plt
import xml.etree.ElementTree as et
import argparse
import time
from cmf.cnimf import CNIMF
from matplotlib import pylab as plt
%matplotlib inline

In [2]:
convolution_width = 3
true_n_components = 4

In [3]:
parser = argparse.ArgumentParser(description='structure detection test')
parser.add_argument('-s', '--seed_number', \
                    action='store', \
                    nargs='?', \
                    const=None, \
                    default=0, \
                    type=int, \
                    choices=None, \
                    help='seed_number', \
                    metavar=None)
parser.add_argument('-d', '--dat_dir', \
                    action='store', \
                    nargs='?', \
                    const=None, \
                    default='../dat/structure_detection', \
                    type=str, \
                    choices=None, \
                    help='Directory where npy files will be stored.', \
                    metavar=None)
parser.add_argument('-n', '--dat_npz', \
                    action='store', \
                    nargs='?', \
                    const=None, \
                    default='structure_result_{}_{}_{}.npz'.format(convolution_width, true_n_components, time.strftime('%Y%m%d%H%M')), \
                    type=str, \
                    choices=None, \
                    help='npz file name', \
                    metavar=None)


_StoreAction(option_strings=['-n', '--dat_npz'], dest='dat_npz', nargs='?', const=None, default='structure_result_3_4_201705131528.npz', type=<class 'str'>, choices=None, help='npz file name', metavar=None)

In [4]:
args = parser.parse_args([])

In [5]:
seed(args.seed_number)


In [6]:
n_tests = 50
n_criteria = 5
n_samples_list = [100, 200, 300, 400, 500, 600, 700, 800]
missing_rate_list = [0.2, 0.6]
data_dim = 12
convolution_width = convolution_width
true_n_components_list = [true_n_components]
gamma_shape = 2.0
gamma_rate = 2.0
base_max = 10.0
best_n_bases = np.zeros([len(true_n_components_list), len(n_samples_list), n_tests, n_criteria])
completion_error = np.zeros([len(true_n_components_list), len(missing_rate_list), len(n_samples_list), n_tests, n_criteria])
accuracy = np.zeros([len(true_n_components_list), len(n_samples_list), n_criteria])

In [7]:
# %pdb
# import warnings
# warnings.filterwarnings('error')
for i_n_components in range(len(true_n_components_list)):
    true_n_components = true_n_components_list[i_n_components]
    for i_n_samples in range(len(n_samples_list)):
        n_samples = n_samples_list[i_n_samples]
        print('n_samples', n_samples)
        for i_test in range(n_tests):
            print('i_test', i_test)
            true_activation = np.random.gamma(gamma_shape, 1.0 / gamma_rate, [n_samples, true_n_components])
            true_base = np.random.uniform(0.0, base_max, [convolution_width, true_n_components, data_dim])
            X = np.random.poisson(CNIMF.convolute(true_activation, true_base))
            arg_dict = dict(
                convolution_max = convolution_width,
                component_max = X.shape[1],
                true_width = convolution_width,
                gamma_shape = gamma_shape,
                gamma_rate = gamma_rate,
                base_max = base_max,
                convergence_threshold = 0.00001,
                loop_max = 1000)
            factorizer = CNIMF(**arg_dict)
            filtre = np.ones(X.shape)
            factorizer.fit(X, None, filtre)
            for i_criterion in range(n_criteria):
                best_n_bases[i_n_components, i_n_samples, i_test, i_criterion] = factorizer.criteria[i_criterion].best_structure[1]
            for i_missing_rate in range(len(missing_rate_list)):
                missing_rate = missing_rate_list[i_missing_rate]
                print(missing_rate)
                factorizer = CNIMF(**arg_dict)
                filtre = np.random.binomial(1, missing_rate, X.shape)
                factorizer.fit(X, None, filtre)
                for i_criterion in range(n_criteria):
                    completion_error[i_n_components, i_missing_rate, i_n_samples, i_test, i_criterion] = factorizer.criteria[i_criterion].completion_error/np.prod(X.shape)
                print(completion_error[i_n_components, i_missing_rate, i_n_samples, i_test, :])
    accuracy[i_n_components, :, :] = np.mean(best_n_bases[i_n_components, :, :, :]==true_n_components, axis = 1)

n_samples 100
i_test 0
0.2
[ 2.59187969  2.11474033  2.11474033  2.11474033  2.11474033]
0.6
[ 0.83806144  0.6857172   0.6857172   0.6857172   0.72473348]
i_test 1
0.2
[ 2.22682791  1.97540224  1.97540224  1.97540224  1.97540224]
0.6
[ 0.83314489  0.76470901  0.76470901  0.76470901  0.71732095]
i_test 2
0.2
[ 2.71646154  3.03583396  2.71646154  3.03583396  3.03583396]
0.6
[ 0.99732539  0.85432028  0.85432028  0.85432028  0.87651289]
i_test 3
0.2
[ 1.94303569  1.8265866   1.8265866   1.8265866   1.8265866 ]
0.6
[ 0.82268097  0.91622625  0.91622625  0.91622625  0.81695051]
i_test 4
0.2
[ 2.00852151  2.39974426  2.39974426  2.39974426  2.39974426]
0.6
[ 0.82538037  0.758657    0.758657    0.758657    0.76497825]
i_test 5
0.2
[ 2.07439917  2.65748885  2.65748885  2.65748885  2.65748885]
0.6
[ 0.83591629  0.77165367  0.77165367  0.77165367  0.70879073]
i_test 6
0.2
[ 2.54662356  2.13373696  2.13373696  2.13373696  2.13373696]
0.6
[ 0.78121993  0.74338071  0.74338071  0.74338071  0.79176992]

In [8]:
np.savez(args.dat_dir + '/' + args.dat_npz,
         accuracy=accuracy,
         completion_error=completion_error,
         best_n_bases=best_n_bases,
         n_samples_list = n_samples_list,
         missing_rate_list = missing_rate_list, 
         true_n_components_list = true_n_components_list,
         convolution_width = convolution_width)

In [9]:
# npz = np.load(args.dat_dir + '/artificial_result_201701210117.npz')

In [10]:
accuracy

array([[[ 0.  ,  0.58,  0.36,  0.78,  0.02],
        [ 0.  ,  0.06,  0.  ,  0.14,  0.46],
        [ 0.  ,  0.  ,  0.  ,  0.02,  0.6 ],
        [ 0.  ,  0.  ,  0.  ,  0.  ,  0.4 ],
        [ 0.  ,  0.  ,  0.  ,  0.  ,  0.04],
        [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
        [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ],
        [ 0.  ,  0.  ,  0.  ,  0.  ,  0.  ]]])

In [11]:
completion_error

array([[[[[ 2.59187969,  2.11474033,  2.11474033,  2.11474033,  2.11474033],
          [ 2.22682791,  1.97540224,  1.97540224,  1.97540224,  1.97540224],
          [ 2.71646154,  3.03583396,  2.71646154,  3.03583396,  3.03583396],
          ..., 
          [ 2.12995231,  2.04260333,  2.04260333,  2.04260333,  2.04260333],
          [ 2.03420448,  2.33831089,  2.33831089,  2.33831089,  2.33831089],
          [ 2.0827489 ,  1.81642151,  1.81642151,  1.81642151,  1.81642151]],

         [[ 2.09017807,  2.32587514,  2.32587514,  1.79346669,  1.79346669],
          [ 1.87918371,  2.30249302,  2.30249302,  2.30249302,  1.75711891],
          [ 2.12699644,  2.12699644,  2.12699644,  1.86346543,  1.86346543],
          ..., 
          [ 1.94524786,  2.24663377,  2.24663377,  1.5869845 ,  1.5869845 ],
          [ 2.16407678,  2.16407678,  2.16407678,  2.16407678,  2.02965032],
          [ 1.70936549,  1.70936549,  1.70936549,  1.48464604,  1.48464604]],

         [[ 1.85392281,  2.01164131,  2.